# Possible improvements to Vamb

Vamb is under an MIT licence, so please feel free to fork, extend, or copy Vamb.

Here's a short wish list for improvements to Vamb we haven't had time for:

<a id='better_exploitation_of_tnf'></a>

__Better exploitation of TNF information in clustering__

There's quite a lot of information in the TNF of contigs, and Vamb is fairly bad at exploiting it. 

The expected TNF distance between two contigs follows approximately a chi distribution (there's some theoretical argument for this, but more importantly, it really empirically does). Similarly, the empirical probability of two contigs belonging to different species as a function of the TNF is well modeled by the cumulative density function of a chi distribution.

However, the exact shape of the chi distribution depends on the phylogenetic distance between the genomes of the contigs, and also the the lengths of the two contigs. Hence, a clustering algorithm which simply looks at the raw TNF value without taking in to account contig lengths is not very effective.

When experimenting with Vamb, we checked if we could, when presented with random pairs of contigs, heurisitcally estimate the parameters of the estimated chi distribution of TNF distances between contigs of those lengths and based on that distribution predict whether or not the two contigs belonged in the same bin. It was quite accurate, but instantiating a `scipy.stats.chi` object with the right parameters for each contig pair would make our clustering algorithm take weeks or months to run for a one-million-contig dataset.

A possible future approach could be to encode the depths and TNF independently with the VAE (although it could still train using both depths and TNF at the same time), and, when calculating the contig-contig distances during clustering, using a heuristic approximation of the chi distribution which can be computed quickly. Alternatively, one could cluster exclusively on depths, then afterwards identify contigs in bins with divergent TNF and/or recruit unbinned contigs to bins with similar TNF.

<a id='switch_to_true_distance_measure'></a>

__Switch to a true distance measure for tandem clustering__

There is a current problem in the implementation of tandem clustering. To explain it, let me begin at the beginning:

There is a problem with the regular clustering algorithm in that it scales O(n<sup>2</sup>), and so becomes painfully slow with larger datasets. We can get around this by splitting the dataset in several partitions, and clustering each partition.

But this creates another problem: When splitting the dataset, it's highly likely that the contigs of a true bin will end up in different partitions. If that happens, it's impossible for the bin to be reconstructed. And we can't easily partition in a manner which preserves the bins' integrity, because that would require us to know the bins beforehand which is the entire point of clustering!

We can solve it with the following splitting function:

    function split(set contigset, float INNER, float OUTER):
        while there are more than 20,000 contigs in contigset:
            S = random contig from contigset
            partition = All contigs in contigset within OUTER distance of S
            yield partition
            superfluous = All contigs in contigset within INNER distance of S
            delete superfluous from contigset
            
        yield contigset # last partition with at most 20,000 elements
        
We can prove this works:

    For any bin B, for any partition seed S, let C be the contig in B closest to S
    Let F be the contig in B furthest from S
    If |SC| > INNER, all contigs in B remains in contigset and the bin is not split (1)
    If |SF| ≤ OUTER, all contigs in B is contained in the partition, bin is not split (2)
    Let us assume we have picked values of INNER and OUTER such that OUTER-INNER > |CF| (cond. A)

    If |SC| > INNER:
        Bin B is not split since (1)

    Else it must be the case that:
        |SC| ≤ INNER, which can be rearranged by (A) to
        |SC| ≤ OUTER - |CF|, adding |CF| gives
        |SC| + |CF| ≤ OUTER, and by the tringle inequality |SF| ≤ |SC| + |CF|:
        |SF| ≤ OUTER, which, by (2) means bin B is not split
            
Hence we just need to pick values for `INNER` and `OUTER` to follow condition A, which means that the difference between `INNER` and `OUTER` should be above the cluster diameter for most realistic bins. No problem.

Unfortunately, this relies on the triangle inequality `|SF| ≤ |SC| + |CF|` which does not hold true for Pearson distance and Spearman distance, and so bins will likely get split when partitioning, giving lower recalls. This implies that tandem clustering will perform better with other distance measures like 1-norm or 2-norm distance. Of course, we can't just use 1-norm distance when partitioning and then use Pearson when clustering!

While it would be trivial to change the distance measure used in `clustering.py`, it is difficult to map e.g. 1-norm distances to a fixed interval [0, 1], and so we have not found a way to approximate the clustering threshold needed. Also, preliminary tests we did have shown that Pearson distance separates the latent representation slightly better than 1- or 2-norm distance (although the loss of accurary by poor partitioning might offset this).

<a id='create_better_autoencoder'></a>

__Create a better autoencoder__

We aren't actually machine learning experts, and the details of the VAE implementation has been a combination of "I think that it might work if we...", blind testing ("stochastic grad student descent" is real), and reading lots of machine learning blogs. I bet there's room for improvement if someone with a solid machine learning background and a deeper theoretical knowledge of neural networks took a crack at designing a VAE for this purpose. In fact, we believe one of the strengths of the VAMB approach is that the VAE itself can likely be much better than the one we've implemented.